# Project 1: Prediction of Ellensburg ASOS hourly wind speed

## Baseline Model: One-hour horizon, "onsite" Ellensburg ASOS data only

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.externals import joblib

multiASOS = pd.read_csv("/Users/verene/Documents/UW-ML410/Project1/IEM_ASOS_multiyr_multisite.txt",
                        index_col=1, header=5, usecols=np.arange(0,11), parse_dates=True, na_values='M')
multiASOS.apply(lambda x: pd.to_numeric(x, errors='ignore'))
print multiASOS.head()
print multiASOS.tail()
#Stations: ELN: Ellensburg (target), EAT: Wenatchee/Pangborn, 1QW: Yakima Training, SMP: Stampede Pass

                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2013-07-01 00:53:00     ELN  96.98  55.94  25.51  100.0    8.0    0.0  29.96   
2013-07-01 00:55:00     EAT  93.92  48.92  21.64  110.0    3.0    0.0  29.93   
2013-07-01 01:53:00     ELN  95.00  57.02  28.18   80.0    6.0    0.0  29.96   
2013-07-01 01:55:00     EAT  93.02  51.08  24.12  120.0    4.0    0.0  29.92   
2013-07-01 02:53:00     ELN  89.06  60.08  37.85   90.0    6.0    0.0  29.95   

                       mslp   vsby  
valid                               
2013-07-01 00:53:00  1013.0   10.0  
2013-07-01 00:55:00  1012.4   10.0  
2013-07-01 01:53:00  1012.7   10.0  
2013-07-01 01:55:00  1012.2   10.0  
2013-07-01 02:53:00  1012.8   10.0  
                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2016-06-30 23:50:00 

In [153]:
# Data exploration
#Save off Ellensburg ASOS for baseline model
EburgASOS = multiASOS[multiASOS['station']=='ELN'].copy()
#Regularize data into hourly means
EburgASOS = EburgASOS.resample('H', closed='right', label='left').mean()

#Determine the expected number of rows for the period of record and compare with actual
dates = pd.date_range(EburgASOS.index[0], EburgASOS.index[-1]+np.timedelta64(seconds=3600), freq='H')
print EburgASOS.shape, len(dates)
#Make sure start and end dates match
print dates[0], EburgASOS.index[0], dates[-1], EburgASOS.index[-1]

(26304, 9) 26304
2013-07-01 00:00:00 2013-07-01 00:00:00 2016-06-30 23:00:00 2016-06-30 23:00:00


In [154]:
#No missing records, so...
#Save off target
EburgWS = pd.Series(EburgASOS[' sknt'].values, index=EburgASOS.index, name='EburgTargetWS')
print EburgWS.head()
#Create a single lag by shifting ASOS timestamp forward two places; timestamp is now two hours ahead of data
print EburgASOS.head()
lag=2 #hours
EburgASOS.index = EburgASOS.index.shift(lag)
print EburgASOS.head()

valid
2013-07-01 00:00:00    8.0
2013-07-01 01:00:00    6.0
2013-07-01 02:00:00    6.0
2013-07-01 03:00:00    6.0
2013-07-01 04:00:00    8.0
Freq: H, Name: EburgTargetWS, dtype: float64
                      tmpf   dwpf   relh   drct   sknt   p01i   alti    mslp  \
valid                                                                          
2013-07-01 00:00:00  96.98  55.94  25.51  100.0    8.0    0.0  29.96  1013.0   
2013-07-01 01:00:00  95.00  57.02  28.18   80.0    6.0    0.0  29.96  1012.7   
2013-07-01 02:00:00  89.06  60.08  37.85   90.0    6.0    0.0  29.95  1012.8   
2013-07-01 03:00:00  82.04  55.04  39.53   80.0    6.0    0.0  29.96  1013.0   
2013-07-01 04:00:00  75.02  55.04  49.81   40.0    8.0    0.0  29.97  1013.6   

                      vsby  
valid                       
2013-07-01 00:00:00   10.0  
2013-07-01 01:00:00   10.0  
2013-07-01 02:00:00   10.0  
2013-07-01 03:00:00   10.0  
2013-07-01 04:00:00   10.0  
                      tmpf   dwpf   relh   drct   

In [155]:
#Join predictors and target again (this will also drop first two rows of EburgWS, and last two rows of EburgASOS)
objs = [EburgASOS, EburgWS]
EburgASOS = pd.concat(objs, join='inner', axis=1)
print EburgASOS.head()

                      tmpf   dwpf   relh   drct   sknt   p01i   alti    mslp  \
valid                                                                          
2013-07-01 02:00:00  96.98  55.94  25.51  100.0    8.0    0.0  29.96  1013.0   
2013-07-01 03:00:00  95.00  57.02  28.18   80.0    6.0    0.0  29.96  1012.7   
2013-07-01 04:00:00  89.06  60.08  37.85   90.0    6.0    0.0  29.95  1012.8   
2013-07-01 05:00:00  82.04  55.04  39.53   80.0    6.0    0.0  29.96  1013.0   
2013-07-01 06:00:00  75.02  55.04  49.81   40.0    8.0    0.0  29.97  1013.6   

                      vsby  EburgTargetWS  
valid                                      
2013-07-01 02:00:00   10.0            6.0  
2013-07-01 03:00:00   10.0            6.0  
2013-07-01 04:00:00   10.0            8.0  
2013-07-01 05:00:00   10.0            3.0  
2013-07-01 06:00:00   10.0            3.0  


In [5]:
#Examine number of missing records from each column
print EburgASOS.isnull().sum(axis=0)

tmpf              213
 dwpf             367
 relh             367
 drct            1237
 sknt             316
 p01i             157
 alti             153
 mslp             263
 vsby             653
EburgTargetWS     316
dtype: int64


In [156]:
#Drop rows with missing values
print EburgASOS.shape
EburgASOS.dropna(axis=0, inplace=True)
print EburgASOS.shape
print EburgASOS.tail()

(26302, 10)
(24349, 10)
                        tmpf     dwpf       relh        drct       sknt  \
valid                                                                     
2016-06-30 19:00:00  75.3500  46.6850  36.190833  310.000000  20.166667   
2016-06-30 20:00:00  79.1060  46.3640  31.558000  314.000000  21.400000   
2016-06-30 21:00:00  81.6800  44.8700  27.420000  308.333333  19.666667   
2016-06-30 22:00:00  82.4675  44.9825  26.833750  307.142857  19.857143   
2016-06-30 23:00:00  84.1550  44.0150  24.482500  303.333333  21.666667   

                      p01i      alti    mslp   vsby  EburgTargetWS  
valid                                                               
2016-06-30 19:00:00    0.0  30.05000  1016.3   10.0      19.666667  
2016-06-30 20:00:00    0.0  30.04100  1016.1   10.0      19.857143  
2016-06-30 21:00:00    0.0  30.03375  1015.6   10.0      21.666667  
2016-06-30 22:00:00    0.0  30.02250  1015.2   10.0      20.125000  
2016-06-30 23:00:00    0.0  30.01000

In [7]:
#Use first two years for training, third year for testing
print EburgASOS[EburgASOS.index<'2015-07-01'].shape
threshidx = EburgASOS[EburgASOS.index<'2015-07-01'].shape[0]
print threshidx

(16063, 10)
16063


In [8]:
#Split dataframe into X and y, train and test
ws_train = EburgASOS.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = EburgASOS.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)

#Exclude target (and timestamp) from predictor matrix
X_train = EburgASOS.ix[:threshidx].as_matrix(EburgASOS.columns[EburgASOS.columns != ('EburgTargetWS' or 'valid' or 'station')]).astype(np.float32)
X_test = EburgASOS.ix[threshidx:].as_matrix(EburgASOS.columns[EburgASOS.columns != ('EburgTargetWS' or 'valid' or 'station')]).astype(np.float32)
print type(ws_train[0][0])

<type 'numpy.float32'>


### BASELINE model

In [9]:
# Train baseline SVR
baseParams = {'kernel':['rbf'], 'C':[0.01, 0.1, 1, 10, 100], 'gamma':[0.00001, 0.0001, 0.001, 0.01],
              'epsilon':[0.01, 0.1, 0.2]}
svrmodel = GridSearchCV(SVR(), baseParams, cv=5, iid=False)
svrmodel.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel.predict(X_test)

# Calculate baseline performance metrics
baseMAE = metrics.mean_absolute_error(np.ravel(ws_test), ws_hat)
baseRMSE = np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
baseBias = ws_hat.mean() - ws_test.mean()
    #This seems like a generous way to calculate bias, but it seems to be the standard in my industry.
print baseMAE, baseRMSE, baseBias, svrmodel.best_params_

### 2.6514939934 3.61144114891 -0.185210889697 {'epsilon': 0.2, 'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

In [10]:
#Save off trained model, so we don't have to train every time the scripts are run
joblib.dump(svrmodel, 'baseline.pkl')

#Load saved trained model when re-running
#svrmodel = joblib.load('baseline.pkl')

## Multi-lagged Model

In [157]:
#Save off Ellensburg ASOS for baseline model
EburgMultiLag = multiASOS[multiASOS['station']=='ELN'].copy()
print EburgMultiLag.head()

#Regularize data into hourly means
EburgMultiLag = EburgMultiLag.resample('H', closed='right', label='left').mean()

#Save off target
EburgWS = pd.Series(EburgMultiLag[' sknt'].values, index=EburgMultiLag.index, name='EburgTargetWS')

numcols = EburgMultiLag.shape[1]
print numcols

#Create multiple lags by shifting ASOS timestamp forward two, three, and four hours
print EburgMultiLag.head()
lags=[2,3,4]
objs=[]
for i in lags:
    df = EburgMultiLag.copy()
    df.index = df.index.shift(i)
    objs.append(df)
objs.append(EburgWS)
EburgMultiLag = pd.concat(objs, join='outer', axis=1)

#Rename columns
for i in lags:
    for j in range(numcols):
        EburgMultiLag.columns.values[(i-2)*numcols+j] = EburgMultiLag.columns.values[(i-2)*numcols+j]+str(i)
print EburgMultiLag.head()

                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2013-07-01 00:53:00     ELN  96.98  55.94  25.51  100.0    8.0    0.0  29.96   
2013-07-01 01:53:00     ELN  95.00  57.02  28.18   80.0    6.0    0.0  29.96   
2013-07-01 02:53:00     ELN  89.06  60.08  37.85   90.0    6.0    0.0  29.95   
2013-07-01 03:53:00     ELN  82.04  55.04  39.53   80.0    6.0    0.0  29.96   
2013-07-01 04:53:00     ELN  75.02  55.04  49.81   40.0    8.0    0.0  29.97   

                       mslp   vsby  
valid                               
2013-07-01 00:53:00  1013.0   10.0  
2013-07-01 01:53:00  1012.7   10.0  
2013-07-01 02:53:00  1012.8   10.0  
2013-07-01 03:53:00  1013.0   10.0  
2013-07-01 04:53:00  1013.6   10.0  
9
                      tmpf   dwpf   relh   drct   sknt   p01i   alti    mslp  \
valid                                                                          
2013-07-01 00:00:0

In [158]:
#Drop rows with any missing data
EburgMultiLag.dropna(axis=0, inplace=True)
print EburgMultiLag.shape

#Use first two years for training, third year for testing
threshidx = EburgMultiLag[EburgMultiLag.index<'2015-07-01'].shape[0]

#Split data frame into X and y, train and test
ws_train = EburgMultiLag.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = EburgMultiLag.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)

#Exclude target (and timestamp) from predictor matrix
X_train = EburgMultiLag.ix[:threshidx].as_matrix(EburgMultiLag.columns[EburgMultiLag.columns != ('EburgTargetWS' or 'valid' or 'station')]).astype(np.float32)
X_test = EburgMultiLag.ix[threshidx:].as_matrix(EburgMultiLag.columns[EburgMultiLag.columns != ('EburgTargetWS' or 'valid' or 'station')]).astype(np.float32)

(22512, 28)


In [13]:
# Train new SVR (same parameters) with multiple lags
parameters = {'kernel':['rbf'], 'C':[0.001, 0.01, 0.1, 1, 10, 100], 'gamma':[0.00001, 0.0001, 0.001, 0.01], 'epsilon':[0.01, 0.1, 0.2]}
svrmodel_multilag = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_multilag.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_multilag.predict(X_test)

# Calculate new performance metrics
multiLagMAE = metrics.mean_absolute_error(ws_test, ws_hat)
multiLagRMSE = np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
multiLagBias = ws_hat.mean() - ws_test.mean()
print multiLagMAE, multiLagRMSE, multiLagBias

### 2.59234462191 3.52985508177 -0.115684936675

In [14]:
#from sklearn.externals import joblib
#joblib.dump(svrmodel_multilag, 'multilag.pkl') 

In [15]:
print svrmodel.best_params_
###{'epsilon': 0.2, 'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

## 3. Multi-lagged Model Using Nearby Offsite Observations

In [16]:
#Reshape so station data are in separate columns
ASOSall = multiASOS.pivot(multiASOS.index, columns='station')
#Transform to hourly data
ASOSall = ASOSall.resample('H', closed='right', label='left').mean()
#Determine the expected number of rows for the period of record and compare with actual
dates = pd.date_range(ASOSall.index[0], ASOSall.index[-1]+np.timedelta64(seconds=3600), freq='H')
print ASOSall.shape, len(dates)
#Make sure start and end dates match
print dates[0], ASOSall.index[0], dates[-1], ASOSall.index[-1]

(26304, 36) 26304
2013-07-01 00:00:00 2013-07-01 00:00:00 2016-06-30 23:00:00 2016-06-30 23:00:00


In [17]:
#Examine number of missing records from each column
print ASOSall.isnull().sum(axis=0)

       station
tmpf   1QW        21849
       EAT          235
       ELN          213
       SMP        21462
 dwpf  1QW        21849
       EAT          235
       ELN          367
       SMP        21462
 relh  1QW        21849
       EAT          235
       ELN          367
       SMP        21462
 drct  1QW        21849
       EAT         1119
       ELN         1237
       SMP        23741
 sknt  1QW        21849
       EAT          335
       ELN          316
       SMP        21535
 p01i  1QW        21849
       EAT          258
       ELN          157
       SMP        21172
 alti  1QW        21849
       EAT          217
       ELN          153
       SMP        21180
 mslp  1QW        21941
       EAT          341
       ELN          263
       SMP        22012
 vsby  1QW        21849
       EAT          358
       ELN          653
       SMP        22630
dtype: int64


In [18]:
#Missing most of the records from 1QW and SMP. Use data from a different Yakima station instead of 1QW, and from Renton
# municipal airport instead of SMP (sadly; predominant strong wind direction is presumably from the west)
multiASOS2 = pd.read_csv("/Users/verene/Documents/UW-ML410/Project1/ASOS_Renton_Yakima.txt",
                        index_col=1, header=5, usecols=np.arange(0,11), parse_dates=True, na_values='M')
multiASOS2.apply(lambda x: pd.to_numeric(x, errors='ignore'))
print multiASOS2.head()
print multiASOS2.tail()

                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2013-07-01 00:53:00     RNT  91.04  59.00  34.22  320.0    4.0    0.0  29.85   
2013-07-01 00:53:00     YKM  96.98  57.92  27.39  190.0    5.0    0.0  29.91   
2013-07-01 00:55:00     1YT  93.92  54.32  26.42    NaN    5.0    0.0  29.93   
2013-07-01 01:53:00     RNT  91.04  59.00  34.22  320.0    5.0    0.0  29.84   
2013-07-01 01:53:00     YKM  96.98  57.92  27.39  170.0    3.0    0.0  29.90   

                       mslp   vsby  
valid                               
2013-07-01 00:53:00  1011.4   10.0  
2013-07-01 00:53:00  1011.8   10.0  
2013-07-01 00:55:00  1011.5    9.0  
2013-07-01 01:53:00  1011.0   10.0  
2013-07-01 01:53:00  1011.6   10.0  
                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2016-06-30 23:50:00 

In [19]:
#Check for duplicates
print (multiASOS2.groupby([multiASOS2.index, 'station']).size()>1).sum()

2


In [20]:
#Remove duplicate entries
multiASOS2 = multiASOS2.reset_index().drop_duplicates(subset=['valid', 'station'], keep='last').set_index('valid')
print (multiASOS2.groupby([multiASOS2.index, 'station']).size()>1).sum()
print multiASOS2.head()

0
                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2013-07-01 00:53:00     RNT  91.04  59.00  34.22  320.0    4.0    0.0  29.85   
2013-07-01 00:53:00     YKM  96.98  57.92  27.39  190.0    5.0    0.0  29.91   
2013-07-01 00:55:00     1YT  93.92  54.32  26.42    NaN    5.0    0.0  29.93   
2013-07-01 01:53:00     RNT  91.04  59.00  34.22  320.0    5.0    0.0  29.84   
2013-07-01 01:53:00     YKM  96.98  57.92  27.39  170.0    3.0    0.0  29.90   

                       mslp   vsby  
valid                               
2013-07-01 00:53:00  1011.4   10.0  
2013-07-01 00:53:00  1011.8   10.0  
2013-07-01 00:55:00  1011.5    9.0  
2013-07-01 01:53:00  1011.0   10.0  
2013-07-01 01:53:00  1011.6   10.0  


In [21]:
#Reshape so station data are in separate columns
ASOSoffsite = multiASOS2.pivot(multiASOS2.index, columns='station')
#Transform to hourly data
ASOSoffsite = ASOSoffsite.resample('H', closed='right', label='left').mean()
#Determine the expected number of rows for the period of record and compare with actual
print ASOSoffsite.shape, len(dates)
#Make sure start and end dates match
print dates[0], ASOSoffsite.index[0], dates[-1], ASOSoffsite.index[-1]
#Examine number of missing records from each column
print ASOSoffsite.isnull().sum(axis=0)

(26304, 27) 26304
2013-07-01 00:00:00 2013-07-01 00:00:00 2016-06-30 23:00:00 2016-06-30 23:00:00
       station
tmpf   1YT         3083
       RNT          180
       YKM          296
 dwpf  1YT         3083
       RNT          248
       YKM          298
 relh  1YT         3083
       RNT          248
       YKM          298
 drct  1YT         4724
       RNT         1493
       YKM         2058
 sknt  1YT         3077
       RNT          238
       YKM          425
 p01i  1YT         3077
       RNT          116
       YKM          258
 alti  1YT         3083
       RNT          115
       YKM          257
 mslp  1YT        26073
       RNT          166
       YKM          455
 vsby  1YT         3082
       RNT          240
       YKM          273
dtype: int64


In [22]:
#Select out data from stations we're interested in (using raw dataframes; it's easier to select out stations this way)
# Too much data missing from 1YT, use other Yakima station (YKM)
tocat=[]
for stat in ['ELN', 'EAT']:
    tocat.append(multiASOS[multiASOS['station']==stat])
for stat in ['RNT', 'YKM']:
    tocat.append(multiASOS2[multiASOS2['station']==stat])
ASOSonoff = pd.concat(tocat, join='outer', axis=0)
print ASOSonoff.head()

                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2013-07-01 00:53:00     ELN  96.98  55.94  25.51  100.0    8.0    0.0  29.96   
2013-07-01 01:53:00     ELN  95.00  57.02  28.18   80.0    6.0    0.0  29.96   
2013-07-01 02:53:00     ELN  89.06  60.08  37.85   90.0    6.0    0.0  29.95   
2013-07-01 03:53:00     ELN  82.04  55.04  39.53   80.0    6.0    0.0  29.96   
2013-07-01 04:53:00     ELN  75.02  55.04  49.81   40.0    8.0    0.0  29.97   

                       mslp   vsby  
valid                               
2013-07-01 00:53:00  1013.0   10.0  
2013-07-01 01:53:00  1012.7   10.0  
2013-07-01 02:53:00  1012.8   10.0  
2013-07-01 03:53:00  1013.0   10.0  
2013-07-01 04:53:00  1013.6   10.0  


In [23]:
#Reshape so station data are in separate columns
ASOSoff3 = ASOSonoff.pivot(ASOSonoff.index, columns='station')

In [24]:
#Combine multilevel columns into one level
ASOSoff3.columns = ['_'.join(col).strip() for col in ASOSoff3.columns.values]
print ASOSoff3.columns.values

['tmpf_EAT' 'tmpf_ELN' 'tmpf_RNT' 'tmpf_YKM' 'dwpf_EAT' 'dwpf_ELN'
 'dwpf_RNT' 'dwpf_YKM' 'relh_EAT' 'relh_ELN' 'relh_RNT' 'relh_YKM'
 'drct_EAT' 'drct_ELN' 'drct_RNT' 'drct_YKM' 'sknt_EAT' 'sknt_ELN'
 'sknt_RNT' 'sknt_YKM' 'p01i_EAT' 'p01i_ELN' 'p01i_RNT' 'p01i_YKM'
 'alti_EAT' 'alti_ELN' 'alti_RNT' 'alti_YKM' 'mslp_EAT' 'mslp_ELN'
 'mslp_RNT' 'mslp_YKM' 'vsby_EAT' 'vsby_ELN' 'vsby_RNT' 'vsby_YKM']


In [25]:
#Transform to hourly data
ASOSoff3 = ASOSoff3.resample('H', closed='right', label='left').mean()

In [26]:
#Save off target
EburgWS = pd.Series(ASOSoff3['sknt_ELN'].values, index=ASOSoff3.index, name='EburgTargetWS')

numcols = ASOSoff3.shape[1]
print numcols

#Create multiple lags by shifting ASOS timestamp forward two, three, and four hours
print ASOSoff3.head()
lags=[2,3,4]
objs=[]
for i in lags:
    df = ASOSoff3.copy()
    df.index = df.index.shift(i)
    objs.append(df)
objs.append(EburgWS)
ASOSoff3 = pd.concat(objs, join='outer', axis=1)

#Rename columns
for i in lags:
    for j in range(numcols):
        ASOSoff3.columns.values[(i-2)*numcols+j] = ASOSoff3.columns.values[(i-2)*numcols+j]+str(i)
print ASOSoff3.head()

36
                     tmpf_EAT  tmpf_ELN  tmpf_RNT  tmpf_YKM  dwpf_EAT  \
valid                                                                   
2013-07-01 00:00:00     93.92     96.98     91.04     96.98     48.92   
2013-07-01 01:00:00     93.02     95.00     91.04     96.98     51.08   
2013-07-01 02:00:00     91.04     89.06     86.00     93.02     53.06   
2013-07-01 03:00:00     82.94     82.04     82.04     91.04     53.06   
2013-07-01 04:00:00     84.02     75.02     80.06     77.00     51.08   

                     dwpf_ELN  dwpf_RNT  dwpf_YKM  relh_EAT  relh_ELN  \
valid                                                                   
2013-07-01 00:00:00     55.94     59.00     57.92     21.64     25.51   
2013-07-01 01:00:00     57.02     59.00     57.92     24.12     28.18   
2013-07-01 02:00:00     60.08     62.96     60.98     27.60     37.85   
2013-07-01 03:00:00     55.04     64.94     60.08     35.71     39.53   
2013-07-01 04:00:00     55.04     62.96     64.

In [27]:
#Drop rows with missing values
print ASOSoff3.shape
ASOSoff3.dropna(axis=0, inplace=True)
print ASOSoff3.shape

(26308, 109)
(15163, 109)


In [28]:
print (ASOSoff3.groupby([ASOSoff3.index]).size()>1).sum()
print ASOSoff3.ix[:threshidx].head().as_matrix(['EburgTargetWS'])

0
[[ 8.]
 [ 3.]
 [ 3.]
 [ 0.]
 [ 3.]]


In [29]:
#Use first two years for training, third year for testing
threshidx = ASOSoff3[ASOSoff3.index<'2015-07-01'].shape[0]
print threshidx

#Split data frame into X and y, train and test
ws_train = ASOSoff3.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = ASOSoff3.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = ASOSoff3.ix[:threshidx].as_matrix(ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)
X_test = ASOSoff3.ix[threshidx:].as_matrix(ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)

9890


In [30]:
# Train model
svrmodel_multilagOffsite = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_multilagOffsite.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_multilagOffsite.predict(X_test)

In [31]:
joblib.dump(svrmodel_multilagOffsite, 'multilagOffsiteObs.pkl')
###print svrmodel_multilagOffsite.best_params_
### {'epsilon': 0.2, 'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}

## 4. Binned Target Wind Speed

### 4.1 Baseline model with binning

In [32]:
#Number of bins desired:
nbins = 5

In [33]:
bins = np.percentile(EburgASOS['EburgTargetWS'], np.linspace(0,100,nbins+1))
#Save binned wind speed with most recent lag
EburgASOS['targetbin'] = np.digitize(EburgASOS[' sknt'], bins)

threshidx = EburgASOS[EburgASOS.index<'2015-07-01'].shape[0]
#Split data frame into X and y, train and test
ws_train = EburgASOS.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = EburgASOS.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = EburgASOS.ix[:threshidx].as_matrix(EburgASOS.columns[EburgASOS.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)
X_test = EburgASOS.ix[threshidx:].as_matrix(EburgASOS.columns[EburgASOS.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)

# Train model
svrmodel_baselineBin = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_baselineBin.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_baselineBin.predict(X_test)

#Save off trained model
joblib.dump(svrmodel_baselineBin, 'baselineBinned.pkl')

# Calculate baseline performance metrics
print metrics.mean_absolute_error(np.ravel(ws_test), np.ravel(ws_hat))
print np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
print ws_hat.mean() - ws_test.mean()
print svrmodel_baselineBin.best_params_

###2.65156463812
###3.61196138801
###-0.18667098241
###{'epsilon': 0.2, 'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

### 4.2 Multi-lag model with offsite observations, binning the most recent lag of the target wind speed

In [34]:
print ASOSoff3.head()
print np.percentile(ASOSoff3['sknt_ELN2'], np.linspace(0,100,nbins+1))
bins = np.linspace(ASOSoff3['sknt_ELN2'].min(),ASOSoff3['sknt_ELN2'].max(),nbins+1)
print bins

#Save binned wind speed with most recent lag
ASOSoff3['targetbin'] = np.digitize(ASOSoff3['sknt_ELN2'], bins)

#Use first two years for training, third year for testing
threshidx = ASOSoff3[ASOSoff3.index<'2015-07-01'].shape[0]
#Split data frame into X and y, train and test
ws_train = ASOSoff3.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = ASOSoff3.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = ASOSoff3.ix[:threshidx].as_matrix(ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)
X_test = ASOSoff3.ix[threshidx:].as_matrix(ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)

# Train model
svrmodel_multilagOffsiteBin = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_multilagOffsiteBin.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_multilagOffsiteBin.predict(X_test)

#Save off trained model
joblib.dump(svrmodel_multilagOffsiteBin, 'multilagOffsiteObsBinned.pkl')

# Calculate baseline performance metrics
print metrics.mean_absolute_error(ws_test, np.ravel(ws_hat))
print np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
print ws_hat.mean() - ws_test.mean()
print svrmodel_multilagOffsiteBin.best_params_

#2.78818985887
#3.68828419899
#-0.261326624792
#{'epsilon': 0.2, 'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}

                     tmpf_EAT2  tmpf_ELN2  tmpf_RNT2  tmpf_YKM2  dwpf_EAT2  \
valid                                                                        
2013-07-01 04:00:00      91.04      89.06      86.00      93.02      53.06   
2013-07-01 05:00:00      82.94      82.04      82.04      91.04      53.06   
2013-07-01 09:00:00      73.04      69.08      73.94      68.00      51.98   
2013-07-01 10:00:00      75.02      66.02      73.28      66.92      55.94   
2013-07-01 11:00:00      71.96      66.02      71.06      66.92      55.94   

                     dwpf_ELN2  dwpf_RNT2  dwpf_YKM2  relh_EAT2  relh_ELN2  \
valid                                                                        
2013-07-01 04:00:00      60.08      62.96      60.98      27.60      37.85   
2013-07-01 05:00:00      55.04      64.94      60.08      35.71      39.53   
2013-07-01 09:00:00      53.96      62.96      59.00      47.59      58.56   
2013-07-01 10:00:00      55.04      62.42      57.02      51.47

['multilagOffsiteObsBinned.pkl',
 'multilagOffsiteObsBinned.pkl_01.npy',
 'multilagOffsiteObsBinned.pkl_02.npy',
 'multilagOffsiteObsBinned.pkl_03.npy',
 'multilagOffsiteObsBinned.pkl_04.npy',
 'multilagOffsiteObsBinned.pkl_05.npy',
 'multilagOffsiteObsBinned.pkl_06.npy',
 'multilagOffsiteObsBinned.pkl_07.npy',
 'multilagOffsiteObsBinned.pkl_08.npy',
 'multilagOffsiteObsBinned.pkl_09.npy',
 'multilagOffsiteObsBinned.pkl_10.npy',
 'multilagOffsiteObsBinned.pkl_11.npy',
 'multilagOffsiteObsBinned.pkl_12.npy',
 'multilagOffsiteObsBinned.pkl_13.npy',
 'multilagOffsiteObsBinned.pkl_14.npy',
 'multilagOffsiteObsBinned.pkl_15.npy',
 'multilagOffsiteObsBinned.pkl_16.npy',
 'multilagOffsiteObsBinned.pkl_17.npy',
 'multilagOffsiteObsBinned.pkl_18.npy',
 'multilagOffsiteObsBinned.pkl_19.npy',
 'multilagOffsiteObsBinned.pkl_20.npy',
 'multilagOffsiteObsBinned.pkl_21.npy',
 'multilagOffsiteObsBinned.pkl_22.npy',
 'multilagOffsiteObsBinned.pkl_23.npy',
 'multilagOffsiteObsBinned.pkl_24.npy',
 'multi

## 5. Feature Selection: MRMR (using MIC and correlation)

In [68]:
#Save binned wind speed with most recent lag
ASOSoff3['targetbin'] = np.digitize(ASOSoff3['sknt_ELN2'], bins)

#Use first two years for training, third year for testing
threshidx = ASOSoff3[ASOSoff3.index<'2015-07-01'].shape[0]
#Split data frame into X and y, train and test
ws_train = ASOSoff3.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = ASOSoff3.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = ASOSoff3.ix[:threshidx].as_matrix(ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)
X_test = ASOSoff3.ix[threshidx:].as_matrix(ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)

In [77]:
cc = np.corrcoef(X_train, rowvar=0)
print cc.shape

(109, 109)


In [71]:
from minepy import MINE
mine = MINE(alpha=0.6, c=15)
micscores=[]
print ASOSoff3.shape[1], X_train.shape[1]
print ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')].values
for j in range(X_train.shape[1]):
    x = X_train[:,j]
    mine.compute_score(np.ravel(x), np.ravel(ws_train))
    print j, mine.mic(), ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')].values[j]
    micscores.append(mine.mic())
print len(micscores), micscores

110 109
['tmpf_EAT2' 'tmpf_ELN2' 'tmpf_RNT2' 'tmpf_YKM2' 'dwpf_EAT2' 'dwpf_ELN2'
 'dwpf_RNT2' 'dwpf_YKM2' 'relh_EAT2' 'relh_ELN2' 'relh_RNT2' 'relh_YKM2'
 'drct_EAT2' 'drct_ELN2' 'drct_RNT2' 'drct_YKM2' 'sknt_EAT2' 'sknt_ELN2'
 'sknt_RNT2' 'sknt_YKM2' 'p01i_EAT2' 'p01i_ELN2' 'p01i_RNT2' 'p01i_YKM2'
 'alti_EAT2' 'alti_ELN2' 'alti_RNT2' 'alti_YKM2' 'mslp_EAT2' 'mslp_ELN2'
 'mslp_RNT2' 'mslp_YKM2' 'vsby_EAT2' 'vsby_ELN2' 'vsby_RNT2' 'vsby_YKM2'
 'tmpf_EAT3' 'tmpf_ELN3' 'tmpf_RNT3' 'tmpf_YKM3' 'dwpf_EAT3' 'dwpf_ELN3'
 'dwpf_RNT3' 'dwpf_YKM3' 'relh_EAT3' 'relh_ELN3' 'relh_RNT3' 'relh_YKM3'
 'drct_EAT3' 'drct_ELN3' 'drct_RNT3' 'drct_YKM3' 'sknt_EAT3' 'sknt_ELN3'
 'sknt_RNT3' 'sknt_YKM3' 'p01i_EAT3' 'p01i_ELN3' 'p01i_RNT3' 'p01i_YKM3'
 'alti_EAT3' 'alti_ELN3' 'alti_RNT3' 'alti_YKM3' 'mslp_EAT3' 'mslp_ELN3'
 'mslp_RNT3' 'mslp_YKM3' 'vsby_EAT3' 'vsby_ELN3' 'vsby_RNT3' 'vsby_YKM3'
 'tmpf_EAT4' 'tmpf_ELN4' 'tmpf_RNT4' 'tmpf_YKM4' 'dwpf_EAT4' 'dwpf_ELN4'
 'dwpf_RNT4' 'dwpf_YKM4' 'relh_EAT4' 'relh_

In [78]:
for i in range(cc.shape[0]):
    cc[i,i]=0

In [124]:
#Find highest MRMR, save that variable, delete variable with highest correlation
#Repeat until you've found ~10
numvar=15
varnames = ASOSoff3.columns[ASOSoff3.columns != ('EburgTargetWS' or 'valid')].values
cctest = cc.copy()
cctest = abs(cctest)
mictest = micscores[:]
print cctest.shape, len(mictest)

(109, 109) 109


In [125]:
best = []
superf = []
for n in range(numvar):
    todel = []
    #i=0
    micmaxidx = mictest.index(max(mictest))
    print "next highest mic score: " + str(micmaxidx) + ", " + varnames[micmaxidx] + ", value: " + str(mictest[micmaxidx])
    best.append(varnames[micmaxidx])
    # First find highly correlated variables
    for d in range(len(cctest[micmaxidx,:])):
        if (cctest[micmaxidx,d] > 0.9):
            todel.append(d)
            superf.append(varnames[d])
            print "high correlation: " + str(d) + ", " + varnames[d] + ", " +  str(cctest[micmaxidx,d])
    todel.append(micmaxidx)
    #Then delete
    todel.sort(reverse=True)
    for k in todel:
        cctest = np.delete(cctest, cctest[k,:], axis=0) #delete kth row
        cctest = np.delete(cctest, cctest[:,k], axis=1) #delete kth column
        
    mictest = [i for j, i in enumerate(mictest) if j not in todel]
    varnames = [i for j, i in enumerate(varnames) if j not in todel]
    print cctest.shape, len(mictest), cc.shape, len(micscores)
print best
print superf

next highest mic score: 17, sknt_ELN2, value: 0.520831941584
high correlation: 108, targetbin, 0.94007856341
(107, 107) 107 (109, 109) 109
next highest mic score: 52, sknt_ELN3, value: 0.448188904439
(106, 106) 106 (109, 109) 109
next highest mic score: 13, drct_ELN2, value: 0.396733355506
(105, 105) 105 (109, 109) 109
next highest mic score: 86, sknt_ELN4, value: 0.38617034249
(104, 104) 104 (109, 109) 109
next highest mic score: 47, drct_ELN3, value: 0.344080514514
(103, 103) 103 (109, 109) 109
next highest mic score: 81, drct_ELN4, value: 0.292768200218
(102, 102) 102 (109, 109) 109
next highest mic score: 9, relh_ELN2, value: 0.24468576695
(101, 101) 101 (109, 109) 109
next highest mic score: 42, relh_ELN3, value: 0.237773220062
(100, 100) 100 (109, 109) 109
next highest mic score: 43, relh_YKM3, value: 0.227147317136
(99, 99) 99 (109, 109) 109
next highest mic score: 14, sknt_EAT2, value: 0.22491067064
high correlation: 15, sknt_RNT2, 0.989645072139
high correlation: 17, p01i_EAT2

In [147]:
print ASOSonoff.head()
ASOSmrmr = ASOSonoff.pivot(ASOSonoff.index, columns='station')
#Combine multilevel columns into one level
ASOSmrmr.columns = ['_'.join(col).strip() for col in ASOSmrmr.columns.values]
#Transform to hourly data
ASOSmrmr = ASOSmrmr.resample('H', closed='right', label='left').mean()

#Save off target
EburgWS = pd.Series(ASOSmrmr['sknt_ELN'].values, index=ASOSmrmr.index, name='EburgTargetWS')

numcols = ASOSmrmr.shape[1]
print numcols
#Create multiple lags by shifting ASOS timestamp forward two, three, and four hours
print ASOSmrmr.head()
lags=[2,3,4]
objs=[]
for i in lags:
    df = ASOSmrmr.copy()
    df.index = df.index.shift(i)
    objs.append(df)
objs.append(EburgWS)
ASOSmrmr = pd.concat(objs, join='outer', axis=1)

#Rename columns
for i in lags:
    for j in range(numcols):
        ASOSmrmr.columns.values[(i-2)*numcols+j] = ASOSmrmr.columns.values[(i-2)*numcols+j]+str(i)
print ASOSmrmr.head()

best.append('EburgTargetWS')
ASOSmrmr = ASOSmrmr[best]
print ASOSmrmr.shape

                    station   tmpf   dwpf   relh   drct   sknt   p01i   alti  \
valid                                                                          
2013-07-01 00:53:00     ELN  96.98  55.94  25.51  100.0    8.0    0.0  29.96   
2013-07-01 01:53:00     ELN  95.00  57.02  28.18   80.0    6.0    0.0  29.96   
2013-07-01 02:53:00     ELN  89.06  60.08  37.85   90.0    6.0    0.0  29.95   
2013-07-01 03:53:00     ELN  82.04  55.04  39.53   80.0    6.0    0.0  29.96   
2013-07-01 04:53:00     ELN  75.02  55.04  49.81   40.0    8.0    0.0  29.97   

                       mslp   vsby  
valid                               
2013-07-01 00:53:00  1013.0   10.0  
2013-07-01 01:53:00  1012.7   10.0  
2013-07-01 02:53:00  1012.8   10.0  
2013-07-01 03:53:00  1013.0   10.0  
2013-07-01 04:53:00  1013.6   10.0  
36
                     tmpf_EAT  tmpf_ELN  tmpf_RNT  tmpf_YKM  dwpf_EAT  \
valid                                                                   
2013-07-01 00:00:00     93.92  

In [148]:
#drop rows with missing values
print ASOSmrmr.shape
ASOSmrmr.dropna(axis=0, inplace=True)
print ASOSmrmr.shape

#check for duplicates
print (ASOSmrmr.groupby([ASOSmrmr.index]).size()>1).sum()

(26308, 16)
(22656, 16)
0


In [149]:
threshidx = ASOSmrmr[ASOSmrmr.index<'2015-07-01'].shape[0]
print threshidx

#Split data frame into X and y, train and test
ws_train = ASOSmrmr.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = ASOSmrmr.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = ASOSmrmr.ix[:threshidx].as_matrix(ASOSmrmr.columns[ASOSmrmr.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)
X_test = ASOSmrmr.ix[threshidx:].as_matrix(ASOSmrmr.columns[ASOSmrmr.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)

# Train model
svrmodel_mrmr = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_mrmr.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_mrmr.predict(X_test)

# Calculate baseline performance metrics
print metrics.mean_absolute_error(np.ravel(ws_test), np.ravel(ws_hat))
print np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
print ws_hat.mean() - ws_test.mean()
print svrmodel_mrmr.best_params_
joblib.dump(svrmodel_mrmr, 'mrmr.pkl')

#14928
#2.62395628243
#3.55578981144
#-0.26312094925
#{'epsilon': 0.2, 'C': 100, 'gamma': 1e-05, 'kernel': 'rbf'}

14928
2.62395628243
3.55578981144
-0.26312094925
{'epsilon': 0.2, 'C': 100, 'gamma': 1e-05, 'kernel': 'rbf'}


['mrmr.pkl',
 'mrmr.pkl_01.npy',
 'mrmr.pkl_02.npy',
 'mrmr.pkl_03.npy',
 'mrmr.pkl_04.npy',
 'mrmr.pkl_05.npy',
 'mrmr.pkl_06.npy',
 'mrmr.pkl_07.npy',
 'mrmr.pkl_08.npy',
 'mrmr.pkl_09.npy',
 'mrmr.pkl_10.npy',
 'mrmr.pkl_11.npy',
 'mrmr.pkl_12.npy',
 'mrmr.pkl_13.npy',
 'mrmr.pkl_14.npy',
 'mrmr.pkl_15.npy',
 'mrmr.pkl_16.npy',
 'mrmr.pkl_17.npy',
 'mrmr.pkl_18.npy',
 'mrmr.pkl_19.npy',
 'mrmr.pkl_20.npy',
 'mrmr.pkl_21.npy',
 'mrmr.pkl_22.npy',
 'mrmr.pkl_23.npy',
 'mrmr.pkl_24.npy',
 'mrmr.pkl_25.npy',
 'mrmr.pkl_26.npy',
 'mrmr.pkl_27.npy',
 'mrmr.pkl_28.npy',
 'mrmr.pkl_29.npy',
 'mrmr.pkl_30.npy',
 'mrmr.pkl_31.npy',
 'mrmr.pkl_32.npy',
 'mrmr.pkl_33.npy',
 'mrmr.pkl_34.npy',
 'mrmr.pkl_35.npy',
 'mrmr.pkl_36.npy',
 'mrmr.pkl_37.npy',
 'mrmr.pkl_38.npy',
 'mrmr.pkl_39.npy',
 'mrmr.pkl_40.npy',
 'mrmr.pkl_41.npy',
 'mrmr.pkl_42.npy',
 'mrmr.pkl_43.npy',
 'mrmr.pkl_44.npy',
 'mrmr.pkl_45.npy',
 'mrmr.pkl_46.npy',
 'mrmr.pkl_47.npy',
 'mrmr.pkl_48.npy',
 'mrmr.pkl_49.npy',
 'mrmr.

In [150]:
# Train model
parameters={'kernel':['rbf'], 'epsilon':[0.15, 0.2, 0.25], 'C':[10, 50, 100, 125, 200, 500], 'gamma': [1e-5, 5e-5, 1e-6, 1e-7]}
svrmodel_mrmr = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_mrmr.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_mrmr.predict(X_test)

# Calculate baseline performance metrics
print metrics.mean_absolute_error(np.ravel(ws_test), np.ravel(ws_hat))
print np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
print ws_hat.mean() - ws_test.mean()
print svrmodel_mrmr.best_params_

###2.62369241518
###3.5554112305
###-0.27695643144
###{'epsilon': 0.25, 'C': 200, 'gamma': 1e-05, 'kernel': 'rbf'}

2.62369241518
3.5554112305
-0.27695643144
{'epsilon': 0.25, 'C': 200, 'gamma': 1e-05, 'kernel': 'rbf'}


In [151]:
best=best[:10]
ASOSmrmr = ASOSonoff.pivot(ASOSonoff.index, columns='station')
#Combine multilevel columns into one level
ASOSmrmr.columns = ['_'.join(col).strip() for col in ASOSmrmr.columns.values]
#Transform to hourly data
ASOSmrmr = ASOSmrmr.resample('H', closed='right', label='left').mean()

#Save off target
EburgWS = pd.Series(ASOSmrmr['sknt_ELN'].values, index=ASOSmrmr.index, name='EburgTargetWS')

numcols = ASOSmrmr.shape[1]
print numcols
#Create multiple lags by shifting ASOS timestamp forward two, three, and four hours
print ASOSmrmr.head()
lags=[2,3,4]
objs=[]
for i in lags:
    df = ASOSmrmr.copy()
    df.index = df.index.shift(i)
    objs.append(df)
objs.append(EburgWS)
ASOSmrmr = pd.concat(objs, join='outer', axis=1)
#Rename columns

for i in lags:
    for j in range(numcols):
        ASOSmrmr.columns.values[(i-2)*numcols+j] = ASOSmrmr.columns.values[(i-2)*numcols+j]+str(i)
print ASOSmrmr.head()

best.append('EburgTargetWS')
ASOSmrmr = ASOSmrmr[best]
print ASOSmrmr.shape

36
                     tmpf_EAT  tmpf_ELN  tmpf_RNT  tmpf_YKM  dwpf_EAT  \
valid                                                                   
2013-07-01 00:00:00     93.92     96.98     91.04     96.98     48.92   
2013-07-01 01:00:00     93.02     95.00     91.04     96.98     51.08   
2013-07-01 02:00:00     91.04     89.06     86.00     93.02     53.06   
2013-07-01 03:00:00     82.94     82.04     82.04     91.04     53.06   
2013-07-01 04:00:00     84.02     75.02     80.06     77.00     51.08   

                     dwpf_ELN  dwpf_RNT  dwpf_YKM  relh_EAT  relh_ELN  \
valid                                                                   
2013-07-01 00:00:00     55.94     59.00     57.92     21.64     25.51   
2013-07-01 01:00:00     57.02     59.00     57.92     24.12     28.18   
2013-07-01 02:00:00     60.08     62.96     60.98     27.60     37.85   
2013-07-01 03:00:00     55.04     64.94     60.08     35.71     39.53   
2013-07-01 04:00:00     55.04     62.96     64.

In [152]:
#drop rows with missing values
print ASOSmrmr.shape
ASOSmrmr.dropna(axis=0, inplace=True)
print ASOSmrmr.shape

#check for duplicates
print (ASOSmrmr.groupby([ASOSmrmr.index]).size()>1).sum()

threshidx = ASOSmrmr[ASOSmrmr.index<'2015-07-01'].shape[0]
print threshidx

#Split data frame into X and y, train and test
ws_train = ASOSmrmr.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = ASOSmrmr.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = ASOSmrmr.ix[:threshidx].as_matrix(ASOSmrmr.columns[ASOSmrmr.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)
X_test = ASOSmrmr.ix[threshidx:].as_matrix(ASOSmrmr.columns[ASOSmrmr.columns != ('EburgTargetWS' or 'valid')]).astype(np.float32)

# Train model
parameters={'kernel':['rbf'], 'epsilon':[0.1, 0.2, 0.25, 0.3], 'C':[10, 100, 150, 200, 500], 'gamma': [1e-3, 1e-4, 1e-5, 5e-5, 1e-6]}
svrmodel_mrmr = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_mrmr.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_mrmr.predict(X_test)

# Calculate baseline performance metrics
print metrics.mean_absolute_error(np.ravel(ws_test), np.ravel(ws_hat))
print np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
print ws_hat.mean() - ws_test.mean()
print svrmodel_mrmr.best_params_
##(26308, 11)
##(22883, 11)
##0
##15065
##2.63428387087
##3.57070157709
##-0.211807476532
##{'epsilon': 0.3, 'C': 150, 'gamma': 1e-05, 'kernel': 'rbf'}

(26308, 11)
(22883, 11)
0
15065
2.63428387087
3.57070157709
-0.211807476532
{'epsilon': 0.3, 'C': 150, 'gamma': 1e-05, 'kernel': 'rbf'}


## 6. Make persistence model, compare improvement over persistence for each model

In [41]:
EburgASOS = multiASOS[multiASOS['station']=='ELN'].copy()
EburgASOS = EburgASOS.resample('H', closed='right', label='left').mean()
print EburgASOS.head()
EburgPers = pd.concat([EburgASOS[' sknt'], EburgASOS[' sknt']], axis=1)
EburgPers.columns=['EburgTargetWS', 'EburgLagWS']
EburgPers['EburgTargetWS'] = EburgPers['EburgTargetWS'].shift(2)
print EburgPers.head()

                      tmpf   dwpf   relh   drct   sknt   p01i   alti    mslp  \
valid                                                                          
2013-07-01 00:00:00  96.98  55.94  25.51  100.0    8.0    0.0  29.96  1013.0   
2013-07-01 01:00:00  95.00  57.02  28.18   80.0    6.0    0.0  29.96  1012.7   
2013-07-01 02:00:00  89.06  60.08  37.85   90.0    6.0    0.0  29.95  1012.8   
2013-07-01 03:00:00  82.04  55.04  39.53   80.0    6.0    0.0  29.96  1013.0   
2013-07-01 04:00:00  75.02  55.04  49.81   40.0    8.0    0.0  29.97  1013.6   

                      vsby  
valid                       
2013-07-01 00:00:00   10.0  
2013-07-01 01:00:00   10.0  
2013-07-01 02:00:00   10.0  
2013-07-01 03:00:00   10.0  
2013-07-01 04:00:00   10.0  
                     EburgTargetWS  EburgLagWS
valid                                         
2013-07-01 00:00:00            NaN         8.0
2013-07-01 01:00:00            NaN         6.0
2013-07-01 02:00:00            8.0         6.0
2

In [42]:
EburgPers.dropna(axis=0, inplace=True)
#Use first two years for training, third year for testing
threshidx = EburgPers[EburgPers.index<'2015-07-01'].shape[0]
#Split data frame into X and y, train and test
ws_train = EburgPers.ix[:threshidx].as_matrix(['EburgTargetWS']).astype(np.float32)
ws_test = EburgPers.ix[threshidx:].as_matrix(['EburgTargetWS']).astype(np.float32)
#Exclude target (and timestamp) from predictor matrix
X_train = EburgPers.ix[:threshidx].as_matrix(['EburgLagWS']).astype(np.float32)
X_test = EburgPers.ix[threshidx:].as_matrix(['EburgLagWS']).astype(np.float32)

# Train model
svrmodel_persistence = GridSearchCV(SVR(), parameters, cv=5, iid=False)
svrmodel_persistence.fit(X_train, np.ravel(ws_train))
ws_hat = svrmodel_persistence.predict(X_test)

# Calculate baseline performance metrics
print metrics.mean_absolute_error(ws_test, np.ravel(ws_hat))
print np.sqrt(metrics.mean_squared_error(ws_test, ws_hat))
print ws_hat.mean() - ws_test.mean()
print svrmodel_persistence.best_params_

###2.82381652422
###3.80478460183
###0.0558128425815
###{'epsilon': 0.1, 'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

2.82381652422
3.80478460183
0.0558128425815
{'epsilon': 0.1, 'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
